In [1]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as mafl



/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:976: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


In [109]:

model_afolu = mafl.AFOLU(sa.model_attributes)

field_abbreviation_subsector = "abbreviation_subsector"
field_category = "subsector_category"
field_information = "information"
field_model_variable = "model_variable"
field_notes = "notes"
field_sector = "sector"
field_subsector = "subsector"
field_units = "unit"
field_variable = "variable"
field_variable_type = "variable_type"

# initialize using sector and modvar
df_out = np.concatenate([np.array(
    list(
        zip([k for x in v], v)
    )
) for k, v in sa.model_attributes.dict_model_variables_by_subsector.items()])

df_out = pd.DataFrame(df_out, columns = [field_subsector, field_model_variable])


# get model variables component
df_modvar = np.concatenate([np.array(
    list(
        zip([k for x in v], v)
    )
) for k, v in sa.model_attributes.dict_model_variables_to_variables.items()])

df_modvar = pd.DataFrame(df_modvar, columns = [field_model_variable, field_variable])
df_out = pd.merge(df_out, df_modvar, how = "outer")

# add in subsector/sector information
df_sector_attr = sa.model_attributes.dict_attributes.get("abbreviation_subsector").table[[field_subsector, field_sector, field_abbreviation_subsector]]
df_out = pd.merge(df_out, df_sector_attr, how = "left")



# add in units info
dict_unit_factors = {
    "emission_gas": sa.model_attributes.varchar_str_emission_gas,
    "units_area": sa.model_attributes.varchar_str_unit_area,
    "units_energy": sa.model_attributes.varchar_str_unit_energy,
    "units_length": sa.model_attributes.varchar_str_unit_length,
    "units_mass": sa.model_attributes.varchar_str_unit_mass,
    "units_monetary": sa.model_attributes.varchar_str_unit_monetary,
    "units_power": sa.model_attributes.varchar_str_unit_power,
    "units_volume": sa.model_attributes.varchar_str_unit_volume
}

fields_srt = sorted(list(dict_unit_factors.keys()))
                    
for k in fields_srt:
    unit_factor = dict_unit_factors.get(k)
    new_col = df_out[field_model_variable].apply(
        sa.model_attributes.get_variable_characteristic,
        args = (unit_factor, )
    )
    
    df_out[k] = new_col


"""    
# build I/O categorization

def get_io_table(
    subsector: str, 
    field_io: str = field_variable_type, 
    field_variable: str = field_variable
):
    inputs, outputs = sa.model_attributes.get_input_output_fields([subsector])
    
    df_in = pd.DataFrame({field_variable: inputs})
    df_out = pd.DataFrame({field_variable: outputs})
    
    df_in[field_io] = "input"
    df_out[field_io] = "output"
    
    return pd.concat([df_in, df_out], axis = 0).reset_index(drop = True)

df_io = []
for subsector in sa.model_attributes.all_subsectors:
    df_io.append(get_io_table(subsector))
df_io = pd.concat(df_io, axis = 0).reset_index(drop = True)    
df_out = pd.merge(df_out, df_io, how = "left")
"""

# get information, variable type, and notes
df_vr = []
dict_rnm = {
    "variable": field_model_variable, 
    "variable_type": field_variable_type,
    "information": field_information,
    "notes": field_notes
}
for k in sa.model_attributes.dict_varreqs.keys():
    tab = sa.model_attributes.dict_varreqs.get(k).table
    tab = tab[list(dict_rnm.keys())].rename(columns = dict_rnm)
    
    df_vr.append(tab) if (len(df_vr) == 0) else df_vr.append(tab[df_vr[0].columns])

df_vr = pd.concat(df_vr, axis = 0).reset_index(drop = True)
df_out = pd.merge(df_out, df_vr, how = "left")


# build categories

df_map_cats = df_out[[field_subsector, field_model_variable]].drop_duplicates()
# add in appropriate category
def get_category(
    subsec: str, 
    modvar: str,
    field_modvar: str = field_model_variable,
    field_category: str = field_category
) -> str:
    #df_out[field_subsector].iloc[0]
    vl = sa.model_attributes.build_varlist(subsec, modvar)
    cats = sa.model_attributes.get_variable_categories(modvar)
    
    if (cats == vl):
        cats = [None for x in vl]
    elif cats is not None:
        cats = [None for x in vl] if (len(cats) != len(vl)) else cats
    
    out = dict(zip(vl, cats)) if (cats is not None) else {vl[0]: None}
    
    return out

dict_cats = {}
for i in range(len(df_map_cats)):
    dict_cats.update(
        get_category(df_map_cats[field_subsector].iloc[i], df_map_cats[field_model_variable].iloc[i])   
    )
    
df_out[field_category] = df_out[field_variable].replace(dict_cats)
    

# sort output
df_out = df_out[
    [
        field_variable,
        field_model_variable,
        field_subsector,
        field_abbreviation_subsector,
        field_sector,
        field_category,
        field_variable_type,
    ] + fields_srt + [
        field_information,
        field_notes
    ]
]

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:976: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


In [111]:
?ing.BaseInputDatabase

Object `ing.BaseInputDatabase` not found.


In [110]:
df_out.to_csv(os.path.join(sa.dir_out, "sisepuede_variable_information_table_20221011.csv"), encoding = "UTF-8", index = None)



In [98]:

df_vr = []
dict_rnm = {
    "variable": field_model_variable, 
    "variable_type": field_variable_type,
    "information": field_information,
    "notes": field_notes
}
for k in sa.model_attributes.dict_varreqs.keys():
    tab = sa.model_attributes.dict_varreqs.get(k).table
    tab = tab[list(dict_rnm.keys())].rename(columns = dict_rnm)
    
    df_vr.append(tab) if (len(df_vr) == 0) else df_vr.append(tab[df_vr[0].columns])

df_vr = pd.concat(df_vr, axis = 0).reset_index(drop = True)
    

In [99]:
df_vr

,model_variable,variable_type,information,notes
0,Carrying Capacity Scalar,Input,This scalar represents changes to the number o...,NaN
1,Elasticity of Livestock Demand to GDP per Capita,Input,Elasticity of meat/animal products demand to i...,Some validation of assumption that income and ...
2,Fraction Nitrogen Excretion in Dung,Input,Fraction of nitrogen :math:`F_{N-DUNG}` excret...,NaN
3,Total Domestic Animal Mass,Output,"Sum, across all livestock animals, of product ...",NaN
4,:math:\text{CH}_4 Forest Methane Emissions,Input,Methane emissions from forests,See `Yale <https://e360.yale.edu/features/scie...
...,...,...,...,...
521,:math:\text{CH}_4 Emissions from SCOE,Output,NaN,NaN
522,:math:\text{CO}_2 Emissions from SCOE,Output,NaN,NaN
523,:math:\text{N}_2\text{O} Emissions from SCOE,Output,NaN,NaN
524,Electrical Energy Demand from SCOE,Output,NaN,none
